In [ ]:
import pydeck as pdk
import requests
from geopy.distance import geodesic
import webbrowser
import os

# Function to get hospitals using OpenStreetMap Overpass API
def get_nearby_hospitals(lat, lon, radius=5000):
    overpass_url = "http://overpass-api.de/api/interpreter"
    query = f"""
    [out:json];
    node["amenity"="hospital"](around:{radius},{lat},{lon});
    out;
    """
    response = requests.get(overpass_url, params={'data': query})
    
    # Check if API request was successful
    if response.status_code != 200:
        print("Error fetching data from Overpass API.")
        return []

    data = response.json()
    
    hospitals = []
    for element in data.get("elements", []):
        hospital_lat = element.get("lat")
        hospital_lon = element.get("lon")
        name = element.get("tags", {}).get("name", "Unknown Hospital")
        distance = geodesic((lat, lon), (hospital_lat, hospital_lon)).meters
        hospitals.append({"name": name, "lat": hospital_lat, "lon": hospital_lon, "distance": distance})
    
    hospitals = sorted(hospitals, key=lambda x: x["distance"])
    
    return hospitals[:5]  # Return the 5 nearest hospitals

# Function to plot hospitals using a location symbol for user and squares for hospitals
def plot_hospitals(user_lat, user_lon, hospitals):
    # Location symbol for user
    location_symbol_url = "https://upload.wikimedia.org/wikipedia/commons/8/88/Map_marker.svg"
    
    # Define markers
    data = [
        {"name": "Your Location", "lat": user_lat, "lon": user_lon, "icon": location_symbol_url, "size": 100}  # User location with location symbol
    ]

    for hospital in hospitals:
        data.append({
            "name": hospital["name"],
            "lat": hospital["lat"],
            "lon": hospital["lon"],
            "color": [0, 255, 0],  # Green for hospitals
            "size": 100  # Bigger size for square markers (you can adjust this as needed)
        })

    # Create an icon layer for user location and scatterplot layer for hospitals
    icon_layer = pdk.Layer(
        "IconLayer",
        data,
        get_position=["lon", "lat"],
        get_icon="icon",
        get_size=6,  # Adjust icon size
        pickable=True,
    )

    # Create a scatterplot layer for hospitals (square markers)
    scatter_layer = pdk.Layer(
        "ScatterplotLayer",
        data,
        get_position=["lon", "lat"],
        get_color="color",
        get_radius="size",  # The radius will determine the size of the square
        pickable=True,
    )

    # ViewState to center map on user location with zoom
    view_state = pdk.ViewState(latitude=user_lat, longitude=user_lon, zoom=14, pitch=0)

    # Create PyDeck deck with the icon layer and scatterplot layer
    r = pdk.Deck(layers=[icon_layer, scatter_layer], initial_view_state=view_state, tooltip={"text": "{name}"})

    # Save the map to an HTML file
    html_file = "map.html"
    r.to_html(html_file, notebook_display=True)
    print(f"Map saved as '{html_file}'.")

    # Try to open automatically
    try:
        webbrowser.open("file://" + os.path.abspath(html_file))
    except:
        print("Could not open map automatically. Please open 'map.html' manually.")

# Example usage
if __name__ == "__main__":
    user_lat = float(input("Enter latitude: "))
    user_lon = float(input("Enter longitude: "))

    nearby_hospitals = get_nearby_hospitals(user_lat, user_lon)

    # Debugging: Print hospital data
    print("\nHospitals Data:", nearby_hospitals)

    if nearby_hospitals:
        print("\nNearest 5 hospitals:")
        for h in nearby_hospitals:
            print(f"{h['name']} - {h['distance']:.2f} meters away")

        plot_hospitals(user_lat, user_lon, nearby_hospitals)
    else:
        print("No hospitals found nearby.")



Hospitals Data: [{'name': 'Shri Vigneshwaran Hospital', 'lat': 11.2833345, 'lon': 78.235441, 'distance': 2903.990292486044}, {'name': 'Sendamangalam Government Hospital', 'lat': 11.27988, 'lon': 78.234922, 'distance': 3129.3361662759494}]

Nearest 5 hospitals:
Shri Vigneshwaran Hospital - 2903.99 meters away
Sendamangalam Government Hospital - 3129.34 meters away
Map saved as 'map.html'.
